In [2]:
import requests
from bs4 import BeautifulSoup

#? List of URLs to scrape
urls = [
    "https://arifbutt.me/",
    "https://www.arifbutt.me/courses/",
    "https://www.arifbutt.me/publications/",
    "https://www.arifbutt.me/research/",
    "https://www.arifbutt.me/cv/",
    "https://www.arifbutt.me/contact/"

]

documents = []

for url in urls:
    response = requests.get(url, verify=False)  # Disable SSL verification
    soup = BeautifulSoup(response.content, 'html.parser')

    #? Extracting text content
    page_text = ' '.join([p.text for p in soup.find_all('p')])
    documents.append({"text": page_text, "url": url})

print("Documents Loaded:", documents)



/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'arifbutt.me'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.arifbutt.me'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/urllib3/connectionpool.py:1061: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.arifbutt.me'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/a

Documents Loaded: [{'text': 'Dr. Muhammad Arif Butt is an Assistant Professor at the Department of Data Science, University of the Punjab (PU), Lahore, Pakistan. He received his MSc and MPhil degrees both with a Gold Medal from PUCIT, University of the Punjab. Dr. Butt also earned his Ph.D. in Computer Science from the same University. His research focuses on applying fuzzy inference models in operating, embedded, and cloud-based systems/services, where decision making is involved under imprecise and vague parameters. His teaching interests are embedded/real time operating systems, system programming, data science and machine learning. His management and teaching experience spans over 33 years in various set ups of Pakistan Army and at University of the Punjab, Lahore, Pakistan. He is a detail-oriented multi-tasker with strong organizational skills, is a tactful team player, thrive within group environment and enjoys a pleasant personality You can explore the exciting educational offer

In [3]:
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Assuming 'documents' is a list of dictionaries with 'text' and 'url' keys
formatted_documents = [
    Document(page_content=doc["text"], metadata={"url": doc["url"]})
    for doc in documents
]

# Load embeddings model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# Create FAISS Index with formatted documents
vector_store = FAISS.from_documents(formatted_documents, embeddings)

/var/folders/ln/3h7vxn1x297c87rv0v1zvhyc0000gn/T/ipykernel_48854/912226109.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
import openai
from IPython.display import display, Markdown

# Set your OpenAI API key
openai.api_key = "key"  # Replace with your OpenAI API key

def rag_chatbot():
    while True:
        user_query = input("User: ")
        if user_query.lower() in ["exit", "quit"]:
            break

        #! Retrieve documents (from your RAG setup)
        retrieved_docs = vector_store.similarity_search(user_query)
        context = " ".join([doc.page_content for doc in retrieved_docs])
        
        #! Create the conversation context
        messages = [
            {"role": "system", "content": "You are a knowledgeable and helpful assistant that retrieve information from the documents I have provided you.."},
            {"role": "user", "content": context + "\nUser: " + user_query}
        ]

        #! Generate response with a higher max_tokens value
        response = openai.ChatCompletion.create(
            model="gpt-4o",
            messages=messages,
            max_tokens=300,  #? Increased to allow for a fuller response
            temperature=0.6,  #? Lowered to make responses more informative
            top_p=0.9
        )

        # Extract and display the response in Markdown format for better presenation 
        bot_response = response['choices'][0]['message']['content'].strip()
        display(Markdown(f"**User:** {user_query}\n\n**Bot:** {bot_response}"))


rag_chatbot()

AuthenticationError: Incorrect API key provided: sk-proj-********************************************************************************************************************************************************1mcA. You can find your API key at https://platform.openai.com/account/api-keys.

In [14]:
%pwd

'/Users/soban/project'